In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (6.4, 4)

In [ ]:
def load_csv(name):
    df = pd.read_csv(name)
    
    # convert from nanos to millis
    timeCols = df.columns != 'ProbTries'
    df.loc[:, timeCols] = df.loc[:, timeCols] / 1_000_000
    
    if 'xy' not in name:
        df['CommitProb'] = df['CommitProbTotal'] / df['ProbTries']
        df['SignProb'] = df['SignProbTotal'] / df['ProbTries']
        df = df.drop(columns=['CommitProbTotal', 'SignProbTotal'])

    return df

In [ ]:
files = [
    'jcop21-x',
    'jcop3-x',
    'jcop3-xy',
    'jcop4-x',
    'jcop4-xy',
    'gd60-x',
    'gd70-x',
]
dfs = {
    file.replace('-', ' ').upper(): load_csv(f'data/{file}.csv')
    for file in files
}
all = pd.concat(dfs)

In [ ]:
# check extreme values
for name, df in dfs.items():
    df.plot.box(subplots=True, figsize=(12, 4), title=name)
    plt.tight_layout()

In [ ]:
# commands kgen, dkgen, and commit on cards using ECDH_XY perform little extra
# computation other than calling the JavaCard-provided algorithms, and thus
# exhibit low time deviations
all.groupby(level=0).std()

In [ ]:
means = all.groupby(level=0).mean()
means

In [ ]:
meansX = means.loc[filter(lambda key: key[-1] == 'X', dfs.keys())]

isSlow = means['Commit'] > 2_000
isSlowX = meansX['Commit'] > 2_000

In [ ]:
palette = sns.color_palette('Paired')
colorScheme = {
    'Dkgen': palette[4],
    'Commit': palette[6],
    'CommitProb': palette[4],
    'Sign': palette[8],
}

In [ ]:
divs = [isSlow, ~isSlow]
cols = ['Dkgen', 'Commit', 'Sign']
fig, axs = plt.subplots(nrows=2, layout='tight', height_ratios=list(map(sum, divs)))

means.loc[divs[0], cols].sort_values(by='Commit').plot.barh(
    ax=axs[0], stacked=True, color=colorScheme, legend=False
)
means.loc[divs[1], cols].sort_index(ascending=False).plot.barh(
    ax=axs[1], stacked=True, xlabel='Time (ms)', color=colorScheme, legend=True
)

fig.savefig('img/basic-time-bar.svg')

In [ ]:
# extra communication round overhead seems to be shadowed by the high data variance
print(means['Commit'] + means['Sign'] - means['SignCommit'])
print(meansX['CommitProb'] + meansX['SignProb'] - meansX['SignCommitProb'])

In [ ]:
# can try up to 3 times on all cards to benefit from probabilistic signing
# expected number of tries is 2
print(meansX['SignCommit'] / meansX['SignCommitProb'])
print((meansX['Commit'] + meansX['Sign']) / (meansX['CommitProb'] + meansX['SignProb']))

In [ ]:
# need to use matplotlib directly for partially stacked bars
def plotProb(ax, data, width=0.25, y_label='Time (ms)', legend=None):
    data = data.sort_values(by='Commit', ascending=False)

    x = np.arange(len(data))

    ax.bar(
        x, data['Commit'], 
        -width, align='edge',
        label='Commit', color=colorScheme['Commit']
    )
    ax.bar(
        x, data['Sign'],
        -width, align='edge', bottom=data['Commit'],
        label='_Sign', color=colorScheme['Sign']
    )

    bottom = np.zeros(len(data))
    for i in range(2):
        ax.bar(
            x, data['CommitProb'], 
            width, align='edge',  bottom=bottom,
            label=i * '_' + 'CommitProb', color = colorScheme['CommitProb']
        )
        bottom = bottom + data['CommitProb']
        ax.bar(
            x, data['Sign'], 
            width, align='edge', bottom=bottom,
            label=i * '_' + 'Sign', color=colorScheme['Sign'],
        )
        bottom = bottom + data['Sign']

    ax.set_ylabel(y_label)
    ax.set_xticks(x, data.index.to_list())
    if legend:
        ax.legend(loc=legend, )
    ax.set_xlim(-2*width, len(data) - 1 + 2*width)


In [ ]:
divs = [isSlowX, ~isSlowX]

fig, axs = plt.subplots(ncols=2, layout='tight', width_ratios=list(map(sum, divs)), figsize=(6.4,4))
plotProb(axs[0], meansX[divs[0]], legend='upper right')
plotProb(axs[1], meansX[divs[1]], y_label=None)

fig.savefig('img/prob-time-bar.svg')
plt.show()

In [ ]:
print(means[['Dkgen', 'Commit', 'CommitProb', 'Sign']].sort_index()
    .style.format(precision=0, na_rep='').to_latex(hrules=True)
)

In [ ]:
dfComp = pd.DataFrame({
    'Normal': meansX['Commit'] + meansX['Sign'],
    'Probabilistic': 2 * (meansX['CommitProb'] + meansX['Sign']),
}).sort_index()
dfComp['Speedup'] = dfComp['Normal'] / dfComp['Probabilistic']
print(dfComp.style
    .format(precision=0, subset=['Normal', 'Probabilistic'])
    .format(precision=1, subset=['Speedup'])
    .to_latex(hrules=True)
)

In [ ]:
probTries = all.ProbTries.dropna()
nBins = probTries.max()
ax = probTries.plot.hist(
    bins=np.arange(nBins+1) + 0.5,
    edgecolor='white', 
    density=True,
    label='Relative Frequency',
    color=palette[6],
)
ax.set_ylabel('Probability')
ax.set_xlabel('Number of Tries')

x = np.arange(nBins) + 1
y = np.power(np.repeat(1/2, nBins), x)
ax.plot(
    x, y, 'o', 
    label='Geometric Distribution, p=1/2',
    color=palette[9],
)

ax.legend()

plt.savefig('img/prob-tries-hist.svg')

In [ ]:
print(meansX.ProbTries.to_frame().sort_index().T
    .style.format(precision=2).to_latex(hrules=True)
)